# A Telegram Bot - Connect 4

First we import the key for our bot. I've placed mine in a `my_env.py` file which has a `get_token()` function that simply returns my bot's access token. If you don't have a token for your bot, follow the tutorial [here](https://github.com/python-telegram-bot/python-telegram-bot/wiki/Extensions-%E2%80%93-Your-first-Bot) to get one set up.

In [1]:
from my_env import get_token
my_token = get_token()

From here, I'm just following the tutorial above. First we need import the `telegram` package from the Python API (oh boy did I underestimate how difficult this would be when you have near-zero understanding of how Anaconda and Jupyter Notebooks interact).

You'll need to install the API package from [here](https://github.com/python-telegram-bot/python-telegram-bot/blob/master/README.rst#installing). If you are able to `import telegram` at this point without error, you can skip the next cell. I was not that fortunate.

This Jupyter Notebook was unable to find the package which I had installed to my Anaconda environment (after both the `pip` method and cloning the entire repo). After reading [this](https://github.com/jupyter/notebook/issues/1524) and [this](https://stackoverflow.com/questions/4383571/importing-files-from-different-folder), I came up with the solution in the following cell. It can be summed up as "copying the Anaconda path to this notebook's path."

Since the Python in my Anaconda environment was able to `import telegram`, I ran `import sys; sys.path` and copied that list to be returned by a function in my `my_env.py`. I then add each item to this notebook's path.

In [2]:
# Skip this if the next cell works just fine for you.
from my_env import get_anaconda_path
path = get_anaconda_path()
import sys
for p in path:
    sys.path.insert(0, p)

In [3]:
import telegram

Woo hoo! It worked! Now we can actually follow the tutorial. To start working with the bot, we need to initialize an `Updater` object with the bot's token and pull the appropraite `Dispather` object from there.

In [4]:
from telegram.ext import Updater
updater = Updater(token=my_token)
dispatcher = updater.dispatcher

Set up a basic logger.

In [5]:
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

Now to begin handling each of the commands my bot supports, of which there are 4 explicit ones:
1. `/start_game` - Initiates setup for 2 players 
2. `/p1` - Player who sends this will be designated as Player 1
3. `/p2` - Same as `/p1`, but for Player 2
4. `/quit` - Allows a player to concede or restart setup

To do this, there's gonna need to be some bookkeeping. Obviously, there also needs to be some way of managing the Connect4 game itself. That will be done through the `Connect4` class in `Connect4.py`.

In [6]:
from Connect4 import Connect4
## Bookkeeping Globals ##
# Universal Values
game = Connect4()
setupHasStarted = False
gameHasStarted = False
P1 = 0
P2 = 1
cur_player = P1
# Player values
players_set = [False, False]
players_id = [0, 0]
players_name = ['', '']
# Keyboard Markup
custom_keyboard = [['1', '2', '3', '4', '5', '6', '7']]
reply_markup = telegram.ReplyKeyboardMarkup(custom_keyboard)
remove_markup = telegram.ReplyKeyboardRemove()

In [9]:
## /start_game ##
def start_game(bot, update):

    chat_id = update.message.chat_id
    text = ''

    # Setup has yet to begin
    if(not(setupHasStarted)):
        setupHasStarted = True
        text = (r'~~~~ Welcome to Connect 4! ~~~~' + '\n'
                r'  Player 1, please select /p1' + '\n'
                r'  Player 2, please select /p2')
    # Setup has begun, but the game has not -> A player still needs to be set
    elif(not(gameHasStarted)):
        if(not(players_set[P1])):
            text = r'Player 1 still needs to be set. Please use /p1 to do so.'
        elif(not(players_set[P2])):
            text = r'Player 2 still needs to be set. Please use /p2 to do so.'
    # Game has started
    else:
        text = 'The game has already started silly goose!'

    bot.send_message(chat_id=chat_id, text=text)

In [10]:
def start_for_real(bot, chat_id):
    gameHasStarted = True
    text = 'Let the games begin!'
    bot.send_message(chat_id=chat_id, text=text)
    text = players_name[P1] + '\'s turn!' + '\n' + game.boardToString()
    bot.send_message(chat_id=chat_id, text=text, reply_markup=reply_markup)

## /p1 ##
def p1(bot, update):

    chat_id = update.message.chat_id
    user = update.message.from_user

    # Player 1 has yet to be set
    if(not(setupHasStarted) or (setupHasStarted and not(players_set[P1]))):
        setupHasStarted = True
        players_set[P1]  = True
        players_id[P1]   = user.id
        players_name[P1] = user.first_name
        text = players_name[P1] + r' has been set as Player 1.'
        bot.send_message(chat_id=chat_id, text=text)
    # Player 1 is set but game has not started
    elif(players_set[P1] and not(gameHasStarted)):
        text = players_name[P1] + r' is already Player 1!'
        bot.send_message(chat_id=chat_id, text=text)
    
    # If both players are set, start the game!
    if(players_set[P2] and not(gameHasStarted)):
        start_for_real(bot, chat_id)
    
## /p2 ##
def p2(bot, update):

    chat_id = update.message.chat_id
    user = update.message.from_user

    # Player 2 has yet to be set
    if(not(setupHasStarted) or (setupHasStarted and not(players_set[P2]))):
        setupHasStarted = True
        players_set[P2]  = True
        players_id[P2]   = user.id
        players_name[P2] = user.first_name
        text = players_name[P2] + r' has been set as Player 2.'
        bot.send_message(chat_id=chat_id, text=text)
    # Player 2 is set but game has not started
    elif(players_set[P2] and not(gameHasStarted)):
        text = players_name[P2] + r' is already Player 2!'
        bot.send_message(chat_id=chat_id, text=text)
    
    # If both players are set, start the game!
    if(players_set[P1] and not(gameHasStarted)):
        start_for_real(bot, chat_id)

In [11]:
def reset_game():
    game.reset()
    setupHasStarted = False
    gameHasStarted = False
    cur_player = 0
    
    players_set = [False, False]
    players_id = [0, 0]
    players_name = ['', '']

## /quit ##
def quit(bot, update):

    chat_id = update.message.chat_id
    text = ''

    if(not(setupHasStarted)):
        text = (r"You can't quit a game that hasn't even started yet..." + '\n'
                r'Use /start_game to being setup.')
    elif(not(gameHasStarted)):
        text = 'Resetting setup.'
        reset_game()
    elif(players_id[P1] == user_id):
        text = players_name[P1] + ' is a quitter! ' + players_name[P2] + ' wins!'
        reset_game()
    elif(players_id[P2] == user_id):
        text = players_name[P2] + ' is a quitter! ' + players_name[P1] + ' wins!'
        reset_game()

    bot.send_message(chat_id=chat_id, text=text, reply_markup=remove_markup)

Now that all the commands are ready, we need to register them with the dispatcher.

In [12]:
from telegram.ext import CommandHandler
start_game_handler  = CommandHandler('start_game', start_game)
p1_handler          = CommandHandler('p1', p1)
p2_handler          = CommandHandler('p2', p2)
quit_handler        = CommandHandler('quit', quit)

dispatcher.add_handler(start_game_handler)
dispatcher.add_handler(p1_handler)
dispatcher.add_handler(p2_handler)
dispatcher.add_handler(quit_handler)

Now the bot will respond to each of the commands. When the game of Connect 4 starts, the player will be able to send messages with the reply keyboard. These will not be formatted as commands, but rather just normal text messages. Luckily, the Python API makes it easy to handle such messages.

In [13]:
def next_player():
    if(cur_player == 1):
        cur_player = 2
    elif(cur_player == 2):
        cur_player = 1

def handle_move(bot, chat_id, cur_player, col):
    
    res = game.placeChip(cur_player, col)
    
    if(res == -1):
        text = "You can't place a chip there! Try again."
        bot.send_message(chat_id=chat_id, text=text)
    elif(res == 0):
        next_player()
        text = players_name[cur_player] + '\'s turn!' + '\n' + game.boardToString()
        bot.send_message(chat_id=chat_id, text=text)
    elif(res == 1):
        text = players_name[cur_player] + ' wins the game!'
        reset_game()
        bot.send_message(chat_id=chat_id, text=text, reply_markup=remove_markup)
    else:
        text = 'Well... it\'s a tie... good job... I guess.'
        reset_game()
        bot.send_message(chat_id=chat_id, text=text, reply_markup=remove_markup)

import string
def place_chip(bot, update):

    text = update.message.text
    chat_id = update.message.chat_id
    user_id = update.message.from_user.id

    if(text in string.digits and int(text) >= 1 and int(text) <= 7):
        if(gameHasStarted):
            if((cur_player == 1 and not(players_id[P1] == user_id)) or
               (cur_player == 2 and not(players_id[P2] == user_id))):
                text = 'It is not your turn!'
                bot.send_message(chat_id=chat_id, text=text)
            else:
                handle_move(bot, chat_id, cur_player, int(text))

Now to register the appropriate method just like with the commands.

In [14]:
from telegram.ext import MessageHandler, Filters
place_chip_handler = MessageHandler(Filters.text, place_chip)
dispatcher.add_handler(place_chip_handler)

And that's that! Everything should be in place for the bot to function at the basic level. it just needs to run!

In [15]:
# Starts the bot on a separate thread
updater.start_polling()

2019-01-03 17:53:44,802 - telegram.ext.dispatcher - ERROR - An uncaught error was raised while processing the update
Traceback (most recent call last):
  File "C:\Users\JLBGr\Anaconda3\envs\py37\lib\site-packages\python_telegram_bot-11.1.0-py3.7.egg\telegram\ext\dispatcher.py", line 301, in process_update
    handler.handle_update(update, self)
  File "C:\Users\JLBGr\Anaconda3\envs\py37\lib\site-packages\python_telegram_bot-11.1.0-py3.7.egg\telegram\ext\commandhandler.py", line 173, in handle_update
    return self.callback(dispatcher.bot, update, **optional_args)
  File "<ipython-input-9-6f9f4ceb9cfa>", line 8, in start_game
    if(not(setupHasStarted)):
UnboundLocalError: local variable 'setupHasStarted' referenced before assignment


In [16]:
# Stops the bot
updater.stop()

Well, look at that, the bot is unable to read the global `setupHasStarted` variable. This is because the dispatcher is only taking the methods themselves as inputs, not all the external references they might have. The solution I'll use (as discussed [here](https://github.com/python-telegram-bot/python-telegram-bot/issues/1002)) is to wrap everything in a class and use `self` references to access the "global" values. This has been done in `Connect4_Bot.py`.

Error logging and intitialization is now in `bot_script.py`, so we can use the following cell to kick everything off!

In [7]:
# Thanks to updater.idle(), we can cleanly interrupt the kernel
# (Ctrl+C in command line, the "Stop"  button in Jupyter Notebook)
# to stop the bot.
exec(open("bot_script.py").read())

2019-01-03 20:47:13,711 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...
